In [10]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd drive/MyDrive/Colab Notebooks/Sebastian

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Sebastian'
/content/drive/My Drive/Colab Notebooks/Sebastian


Stuff for commiting to Github below

In [ ]:
ROOT = '/content/drive"'

In [ ]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'drive/MyDrive/Colab Notebooks/Sebastian' 
# replace with your Github username 
GIT_USERNAME = "ahhuang007" 
# definitely replace with your
GIT_TOKEN = "new github token here"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "Sebastian" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path     

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
#GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
#print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive"/drive/MyDrive/Colab Notebooks/Sebastian


In [ ]:
!git add .

In [5]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

In [6]:
!git commit -m "committing models, need to figure out why my model has worse rewards but does better"

[main aefced7] committing models, need to figure out why my model has worse rewards but does better
 11 files changed, 10 insertions(+), 9 deletions(-)
 rewrite colab.ipynb (95%)
 rewrite gym-seb/gym_seb/envs/__pycache__/seb_env.cpython-37.pyc (78%)
 create mode 100644 random_model_ppo.zip
 rewrite real_model_ppo.zip (90%)
 delete mode 100644 test_model_ddpg.zip
 delete mode 100644 test_model_ppo.zip


Had to reset the remote origin URL here, removed the code since it contained the personal authorization token.

--------

Pulling from github since I needed the viz file

In [15]:
!git pull origin

remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 5), reused 13 (delta 4), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/ahhuang007/Sebastian
   e801e2e..5f67801  main       -> origin/main
Updating e801e2e..5f67801
Fast-forward
 gym-seb/gym_seb.egg-info/PKG-INFO                  |   8 +++----
 .../gym_seb/__pycache__/__init__.cpython-37.pyc    | Bin 351 -> 344 bytes
 .../envs/__pycache__/__init__.cpython-37.pyc       | Bin 287 -> 280 bytes
 .../envs/__pycache__/seb_env.cpython-37.pyc        | Bin 3737 -> 3805 bytes
 gym-seb/gym_seb/envs/seb_env.py                    |  11 +++++++++-
 viz.py                                             |  24 ++++++---------------
 6 files changed, 21 insertions(+), 22 deletions(-)


In [7]:
!git push origin

Counting objects: 17, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (17/17), done.
Writing objects: 100% (17/17), 167.48 KiB | 6.98 MiB/s, done.
Total 17 (delta 7), reused 0 (delta 0)
remote: Resolving deltas: 100% (7/7), completed with 7 local objects.
To https://github.com/ahhuang007/Sebastian.git
   5f67801..aefced7  main -> main


Getting OpenAI Baselines

In [ ]:
!git clone https://github.com/openai/baselines.git

Cloning into 'baselines'...
remote: Enumerating objects: 3627, done.
remote: Total 3627 (delta 0), reused 0 (delta 0), pack-reused 3627
Receiving objects: 100% (3627/3627), 6.46 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.
Checking out files: 100% (199/199), done.


Baselines doesn't come installed, tragic

In [ ]:
cd baselines

/content/drive/MyDrive/Colab Notebooks/Sebastian/baselines


In [ ]:
!pip install -e .

Installing stable_baselines since it might be better

In [3]:
%cd ..

/content/drive/MyDrive/Colab Notebooks


In [12]:
!pip install stable-baselines3[extra]

Installing my custom environment

In [5]:
cd Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [13]:
%cd gym-seb

/content/drive/My Drive/Colab Notebooks/Sebastian/gym-seb


In [14]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/Sebastian/gym-seb
  Attempting uninstall: gym-seb
    Found existing installation: gym-seb 0.0.4
    Can't uninstall 'gym-seb'. No files were found to uninstall.
  Running setup.py develop for gym-seb


Restart runtime here

In [1]:
%cd drive/My Drive/Colab Notebooks/Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [2]:
from stable_baselines3 import DDPG, PPO
import gym

In [3]:
import gym_seb
env = gym.make('seb-v0', max_timesteps = 10000, use_gui = False)

In [4]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True)

resetting environment


In [5]:
from stable_baselines3.ddpg.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

'''
model finished running when "doneness" was achievable - do i need to do this?
I think the model will keep learning until it is done - if it is done but 
has not reached the number of total timesteps, it will reset and learn again
until it has surpassed the total_timesteps and is done again
'''

model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
from stable_baselines3.common.evaluation import evaluate_policy

#performance with random model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
mean_reward=10001.01 +/- 0.0004527027069843506


In [7]:
model.save("random_model_ppo")

In [8]:
env.reset()
#set total_timesteps equal to n_eval_episodes * max_timesteps
model = model.learn(total_timesteps = 200000, n_eval_episodes = 10, log_interval = 10000)

resetting environment
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has flipped over
resetting environment
robot has f

In [9]:
model.save("real_model_ppo")

In [10]:

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
max timesteps reached
resetting environment
mean_reward=134424.12 +/- 15.775752051725105


So, yeah, my algorithm's improving but it still needs to improve a lot more. It would appear that PPO is better than DDPG, so I think I'll be working with PPO from here on out.